#AI LAB-2 TASK



##New Game Concept: Robot Escape
Game Objective:
Navigate a robot from the starting point R to the exit point E on a grid.
The robot must avoid obstacles X that block its path.
The game will display the grid with obstacles and the robot's path to the exit.

In [ ]:
import heapq
import random
import math

# Function to create a grid with a starting point, an exit, and randomly placed obstacles
def create_grid(size):
    grid = [[' ' for _ in range(size)] for _ in range(size)]  # Create an empty grid filled with spaces
    grid[0][0] = 'R'  # 'R' marks the starting point at the top-left corner (0,0)
    grid[size-1][size-1] = 'E'  # 'E' marks the exit at the bottom-right corner (size-1, size-1)

    # Randomly place obstacles in the grid
    num_obstacles = int(size * size * 0.2)  # Place obstacles in approximately 20% of the grid
    grid = add_obstacles(grid, num_obstacles)

    return grid, (size-1, size-1)  # Return the grid and exit position

# Function to add obstacles to the grid
def add_obstacles(grid, num_obstacles):
    size = len(grid)
    for _ in range(num_obstacles):
        obstacle_x, obstacle_y = random.randint(0, size-1), random.randint(0, size-1)
        # Ensure obstacles are not placed on the start 'R' or exit 'E'
        while grid[obstacle_x][obstacle_y] in ['R', 'E']:
            obstacle_x, obstacle_y = random.randint(0, size-1), random.randint(0, size-1)
        grid[obstacle_x][obstacle_y] = 'X'  # Place the obstacle 'X'
    return grid

# Function to check if a position is valid (within bounds and not blocked by an obstacle)
def is_valid_position(grid, x, y):
    size = len(grid)
    return 0 <= x < size and 0 <= y < size and grid[x][y] != 'X'  # Return true if position is valid

# Heuristic function: Calculates the straight-line (Euclidean) distance between the current node and the goal
def heuristic(a, b):
    return math.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)

# A* algorithm function to find the shortest path
def a_star(grid, start, goal):
    size = len(grid)
    # Priority queue to keep track of nodes to explore (using heapq)
    open_list = []
    heapq.heappush(open_list, (0, start))  # Add the start node with priority 0

    # Dictionaries to store the cost to reach each node and the path taken
    g_score = {start: 0}  # Cost from start to current node
    parent = {start: None}  # To reconstruct the path

    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Possible directions: up, down, left, right

    while open_list:
        _, current = heapq.heappop(open_list)  # Get the node with the lowest f(n)

        # If we reach the goal (exit), stop the search
        if current == goal:
            break

        # Explore neighboring positions (up, down, left, right)
        for direction in directions:
            next_x = current[0] + direction[0]  # Calculate next x-coordinate
            next_y = current[1] + direction[1]  # Calculate next y-coordinate
            next_state = (next_x, next_y)  # Form the next state (position)

            # Check if the next position is valid
            if is_valid_position(grid, next_x, next_y):
                tentative_g_score = g_score[current] + 1  # Cost of moving to the next position

                # If the next state has not been explored or we found a cheaper path to it
                if next_state not in g_score or tentative_g_score < g_score[next_state]:
                    g_score[next_state] = tentative_g_score  # Update the cost to reach this state
                    f_score = tentative_g_score + heuristic(next_state, goal)  # Calculate the total cost f(n)
                    heapq.heappush(open_list, (f_score, next_state))  # Add the state to the open list
                    parent[next_state] = current  # Set the current state as the parent of the next state

    # Reconstruct the path from start to goal
    path = []
    current = goal  # Start from the goal (exit) and work backwards
    while current is not None:
        path.append(current)  # Add current position to the path
        current = parent[current]  # Move to the parent of the current state
    path.reverse()  # Reverse the path to start from the beginning (start to goal)

    return path

# Function to print the grid with lines and the path marked
def print_grid_with_path(grid, path):
    grid_with_path = [row.copy() for row in grid]  # Copy the original grid to preserve it
    for (x, y) in path:
        if grid_with_path[x][y] not in ['R', 'E']:  # Don't overwrite start 'R' and exit 'E'
            grid_with_path[x][y] = '*'  # Mark the path with '*'

    # Print the grid with lines to separate cells clearly
    print("\nGrid with Path:")
    print('-' * (len(grid_with_path) * 4 + 1))  # Print top border line
    for row in grid_with_path:
        print('| ' + ' | '.join(row) + ' |')  # Print row with borders between cells
        print('-' * (len(grid_with_path) * 4 + 1))  # Print horizontal line after each row

# Main function to play the game
def robot_escape():
    # Ask the user for grid size
    size = int(input("Enter the grid size (e.g., 6 for a 6x6 grid): "))

    # Create the grid and place the exit
    grid, goal = create_grid(size)

    # Print the initial grid
    print("\nInitial Grid:")
    print_grid_with_path(grid, [])  # Empty path initially

    # Start the search using A* algorithm
    print("\nFinding the path to the exit using A* algorithm...\n")
    path = a_star(grid, (0, 0), goal)  # Perform A* search

    # Print the final grid with the path marked
    print("\nPath to the Exit:")
    print_grid_with_path(grid, path)  # Show the path to the exit

# Run the game
robot_escape()


Enter the grid size (e.g., 6 for a 6x6 grid): 7

Initial Grid:

Grid with Path:
-----------------------------
| R |   |   |   |   |   | X |
-----------------------------
|   | X |   |   |   |   |   |
-----------------------------
|   |   | X |   |   |   |   |
-----------------------------
|   |   |   | X |   | X |   |
-----------------------------
| X | X |   |   |   |   |   |
-----------------------------
|   |   |   | X |   |   |   |
-----------------------------
| X |   |   |   |   |   | E |
-----------------------------

Finding the path to the exit using A* algorithm...


Path to the Exit:

Grid with Path:
-----------------------------
| R | * | * |   |   |   | X |
-----------------------------
|   | X | * | * |   |   |   |
-----------------------------
|   |   | X | * | * |   |   |
-----------------------------
|   |   |   | X | * | X |   |
-----------------------------
| X | X |   |   | * | * |   |
-----------------------------
|   |   |   | X |   | * | * |
---------------------

##How It Works:

###Setup:

 The game initializes a grid with a robot start point R, an exit E, and obstacles X.

###Pathfinding:

 The A* algorithm calculates the shortest path from the robot to the exit, avoiding obstacles.

###Display:

The grid and the path are displayed to the user, showing how the robot can navigate from the start to the exit.
This "Robot Escape" game provides an engaging way to apply pathfinding algorithms and simulate navigation challenges in a grid-based environment.